In [ ]:
%pip install langchain-core langchain-openai langchain-exa

In [9]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["EXA_API_KEY"] = ""

In [8]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_PROJECT"] = ""

In [10]:
from langchain_exa import ExaSearchRetriever

retriever = ExaSearchRetriever(k=3, highlights=True)
documents = retriever.invoke("Meilleur point de vue à Tokyo")
len(documents)

3

In [11]:
documents[0].metadata

{'title': 'Louis Vuitton dévoile la nouvelle façade hypnotique de sa boutique de Tokyo',
 'url': 'https://www.vogue.fr/vogue-hommes/article/louis-vuitton-boutique-tokyo',
 'id': 'ZmMtrrQq9xry5b5Ce8lGXA',
 'score': 0.17380395531654358,
 'published_date': '2021-03-24',
 'author': 'Alexandre Marain',
 'highlights': ['Sans oublier le mobilier tout en courbe signé Morten Stenbaek et Isamu Noguchi, conçu dans un désir de fluidité.  '],
 'highlight_scores': [0.04403303558210609]}

In [12]:
print(documents[1].metadata["highlights"])

['Publié le 11 mars 2021 à 06h00, mis à jour le 11 mars 2021 à 10h14 Temps de Lecture 8 min. Article réservé aux abonnés       Tribune. Koji Minoura, géologue à l’université de Sendai, près de Fukushima, connaît sa région. Amateur de poésie ancienne, il découvre à la fin des années 1980 un poème du Xe\xa0siècle\xa0: «\xa0Nos manches mouillées de larmes/Etaient témoins que notre amour/Durerait encore lorsque à Sue/Par-dessus le mont des Pins/Se brisent les vagues de l’océan. »  Le mont des Pins, à 4\xa0km de la côte, a-t-il subi un tsunami oublié, lors du séisme connu du 11\xa0juillet 869\xa0?']


In [13]:
print(documents[1].metadata["url"])

https://www.lemonde.fr/idees/article/2021/03/11/fukushima-comment-le-japon-s-est-aveugle-devant-la-certitude-annoncee-d-un-desastre_6072680_3232.html


In [17]:
from langchain_exa import ExaSearchRetriever
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda


retriever = ExaSearchRetriever(k=3, highlights=True)

document_prompt = PromptTemplate.from_template("""
<source>
    <url>{url}</url>
    <highlights>{highlights}</highlights>
</source>
""")

document_chain = RunnableLambda(
    lambda document: {
        "highlights": document.metadata["highlights"], 
        "url": document.metadata["url"]
    }
) | document_prompt

retrieval_chain = retriever | document_chain.map() | (lambda docs: "\n".join([i.text for i in docs]))

print(retrieval_chain.invoke("Meilleur endroit pour visiter le Japon"))


<source>
    <url>https://www.vogue.fr/culture/agenda/story/adresses-special-japon-a-paris/4321</url>
    <highlights>["Le principe ? Un massage facial très dynamique qui réveille l’épiderme et les muscles du visage, effectué par des mains expertes, formées chez le Dr Shogo Mochizuki de Tokyo. Soit un protocole qui apporte éclat et tonicité et qui propulse, le temps d’une petite heure, au Pays du Soleil-levant.  31 Avenue George V, 75008 Paris, Tel.01 49 52 70 00      Savourer les fabuleux sushis de L'Abysse  Au rez-de-chaussée du Pavillon Ledoyen, L'Abysse est le nouveau comptoir à sushis du chef triplement étoilé Yannick Alléno. Confiant les rênes à Yasunari Okazaki, tout y est fait à la japonaise et dans les plus grandes règles de l’art."]</highlights>
</source>


<source>
    <url>https://www.vogue.fr/lifestyle/voyages/diaporama/les-adresses-de-lou-doillon-au-japon/41961</url>
    <highlights>["Japon, 〒106-0032 Tokyo, Minato, Roppongi, ５丁目６−２０, Tel.81 3-6328-1960                 L

In [18]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

generation_prompt = ChatPromptTemplate.from_messages([
    ("system", "Vous êtes un assistant de recherche expert. Vous utilisez du contexte en format xml pour répondre aux questions des utilisateurs."),
    ("human", """
Veuillez répondre à la question suivante en vous basant sur le contexte fourni. Veuillez citer vos sources à la fin de votre réponse.:
     
Query: {query}
---
<context>
{context}
</context>
""")
])

llm = ChatOpenAI()

chain = RunnableParallel({
    "query": RunnablePassthrough(),
    "context": retrieval_chain,
}) | generation_prompt | llm

In [16]:
chain.invoke("Meilleurs lieux touristiques en Corée du Sud").content

"Les meilleurs lieux touristiques en Corée du Sud incluent une variété d'attractions allant des villes dynamiques aux paysages naturels magnifiques. Vous pouvez découvrir l'essentiel de la péninsule sud-coréenne en explorant ses villes animées, ses campagnes pittoresques, ses mers et ses montagnes. Pour personnaliser votre voyage, vous pouvez également participer à des activités culturelles, historiques, en lien avec la nature ou la gastronomie. \n\nSources:\n1. Site Web: https://www.coreevoyage.com/\n2. Site Web: https://www.encoreedusud.com\n3. Site Web: https://www.capcoree.fr/"